# Authorization Audit Tool

This notebook provides a comprehensive tool for auditing and managing authorizations in Operaton/Camunda 7.

## Features
- **Query by Resource ID**: Find all authorizations for a specific resource
- **Query by Business Key**: Find authorizations for process instances by business key
- **Find Principals**: Discover which users/groups have access to a resource
- **Filter by User/Group**: View authorizations for specific users or groups
- **Delete Authorizations**: Remove authorizations (with confirmation)

## Authorization Resource Types
| Type | ID | Description |
|------|-----|-------------|
| Application | 0 | Access to applications (Cockpit, Tasklist, Admin) |
| User | 1 | User management |
| Group | 2 | Group management |
| Group Membership | 3 | Group membership management |
| Authorization | 4 | Authorization management |
| Process Definition | 6 | Process definition access |
| Task | 7 | Task access |
| Process Instance | 8 | Process instance access |
| Deployment | 9 | Deployment access |
| Decision Definition | 10 | Decision definition access |
| Tenant | 11 | Tenant management |
| Tenant Membership | 12 | Tenant membership management |
| Batch | 13 | Batch job access |
| Decision Requirements Definition | 14 | DRD access |
| Report | 15 | Report access |
| Dashboard | 16 | Dashboard access |
| Operation Log | 17 | Operation log access |
| System | 18 | System access |
| Historic Task | 19 | Historic task access |
| Historic Process Instance | 20 | Historic process instance access |

## Usage
1. Run all cells in order
2. Use the tabs to switch between different query modes
3. Enter search criteria and click the action button

In [ ]:
# Initialize environment and imports
import operaton
from operaton import Operaton
import json

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Initialize environment (loads API configuration from localStorage)
await operaton.load_env()
print("✅ Environment loaded successfully")

In [ ]:
# Authorization resource type mappings (Camunda 7)
RESOURCE_TYPES = {
    0: "Application",
    1: "User", 
    2: "Group",
    3: "Group Membership",
    4: "Authorization",
    6: "Process Definition",
    7: "Task",
    8: "Process Instance",
    9: "Deployment",
    10: "Decision Definition",
    11: "Tenant",
    12: "Tenant Membership",
    13: "Batch",
    14: "Decision Requirements Definition",
    15: "Report",
    16: "Dashboard",
    17: "Operation Log",
    18: "System",
    19: "Historic Task",
    20: "Historic Process Instance"
}

# Permission mappings
PERMISSIONS = {
    0: "NONE",
    1: "ALL",
    2: "READ",
    4: "UPDATE",
    8: "CREATE",
    16: "DELETE",
    32: "ACCESS",
    64: "READ_TASK",
    128: "UPDATE_TASK",
    256: "CREATE_INSTANCE",
    512: "READ_INSTANCE",
    1024: "UPDATE_INSTANCE",
    2048: "DELETE_INSTANCE",
    4096: "READ_HISTORY",
    8192: "DELETE_HISTORY",
    16384: "TASK_WORK",
    32768: "TASK_ASSIGN",
    65536: "MIGRATE_INSTANCE",
    131072: "SUSPEND",
    262144: "SUSPEND_INSTANCE",
    524288: "UPDATE_VARIABLE",
    1048576: "DELETE_VARIABLE",
    2097152: "UPDATE_HISTORY"
}

def decode_permissions(permission_value):
    """Decode permission integer to list of permission names."""
    if permission_value == 0:
        return ["NONE"]
    if permission_value == 2147483647:  # MAX_INT = ALL
        return ["ALL"]
    
    perms = []
    for bit, name in sorted(PERMISSIONS.items(), reverse=True):
        if bit > 0 and permission_value >= bit:
            if permission_value & bit:
                perms.append(name)
    return perms if perms else [str(permission_value)]

def format_authorization(auth):
    """Format an authorization for display."""
    resource_type = RESOURCE_TYPES.get(auth.get("resourceType"), str(auth.get("resourceType")))
    perms = decode_permissions(auth.get("permissions", [0])[0] if isinstance(auth.get("permissions"), list) else auth.get("permissions", 0))
    
    return {
        "id": auth.get("id", ""),
        "type": resource_type,
        "resourceId": auth.get("resourceId", "*"),
        "userId": auth.get("userId", ""),
        "groupId": auth.get("groupId", ""),
        "permissions": ", ".join(perms) if isinstance(perms, list) else perms
    }

print("✅ Authorization helpers loaded")

In [ ]:
# === WIDGETS ===

# Tab 1: Query by Resource ID
w_resource_id = widgets.Text(
    value='',
    placeholder='Enter resource ID (e.g., process instance ID)',
    description='Resource ID:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

btn_query_resource = widgets.Button(
    description='Query Authorizations',
    button_style='primary',
    icon='search',
    layout=widgets.Layout(width='200px')
)

output_resource = widgets.Output()

# Tab 2: Query by Business Key
w_business_key = widgets.Text(
    value='',
    placeholder='Enter business key',
    description='Business Key:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

btn_find_principals = widgets.Button(
    description='Find Principals',
    button_style='info',
    icon='users',
    layout=widgets.Layout(width='150px')
)

btn_list_auths = widgets.Button(
    description='List Authorizations',
    button_style='primary',
    icon='list',
    layout=widgets.Layout(width='180px')
)

output_business_key = widgets.Output()

# Tab 3: Filter by User/Group
w_filter_business_key = widgets.Text(
    value='',
    placeholder='Enter business key',
    description='Business Key:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

w_filter_user = widgets.Text(
    value='',
    placeholder='Enter user ID (optional)',
    description='User ID:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

w_filter_group = widgets.Text(
    value='',
    placeholder='Enter group ID (optional)',
    description='Group ID:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

btn_filter_auths = widgets.Button(
    description='Filter Authorizations',
    button_style='primary',
    icon='filter',
    layout=widgets.Layout(width='180px')
)

output_filter = widgets.Output()

# Tab 4: Delete Authorizations
w_delete_business_key = widgets.Text(
    value='',
    placeholder='Enter business key',
    description='Business Key:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

w_delete_user = widgets.Text(
    value='',
    placeholder='Enter user ID (optional)',
    description='User ID:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

w_delete_group = widgets.Text(
    value='',
    placeholder='Enter group ID (optional)',
    description='Group ID:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

w_confirm_delete = widgets.Checkbox(
    value=False,
    description='I confirm I want to delete these authorizations',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

btn_preview_delete = widgets.Button(
    description='Preview Deletions',
    button_style='warning',
    icon='eye',
    layout=widgets.Layout(width='160px')
)

btn_delete_auths = widgets.Button(
    description='Delete Authorizations',
    button_style='danger',
    icon='trash',
    layout=widgets.Layout(width='180px'),
    disabled=True
)

output_delete = widgets.Output()

# State for delete preview
delete_state = {
    'authorizations_to_delete': []
}

print("✅ Widgets created")

In [ ]:
# === EVENT HANDLERS ===

def render_authorizations_table(authorizations, title="Authorizations"):
    """Render authorizations as an HTML table."""
    if not authorizations:
        return f"<p>No {title.lower()} found.</p>"
    
    html = [f'<h3>{title} ({len(authorizations)})</h3>']
    html.append('<table style="border-collapse: collapse; width: 100%;">')
    html.append('<tr style="background-color: #4CAF50; color: white;">')
    html.append('<th style="padding: 8px; border: 1px solid #ddd;">ID</th>')
    html.append('<th style="padding: 8px; border: 1px solid #ddd;">Type</th>')
    html.append('<th style="padding: 8px; border: 1px solid #ddd;">Resource ID</th>')
    html.append('<th style="padding: 8px; border: 1px solid #ddd;">User</th>')
    html.append('<th style="padding: 8px; border: 1px solid #ddd;">Group</th>')
    html.append('<th style="padding: 8px; border: 1px solid #ddd;">Permissions</th>')
    html.append('</tr>')
    
    for i, auth in enumerate(authorizations):
        formatted = format_authorization(auth)
        bg = '#f9f9f9' if i % 2 == 0 else 'white'
        html.append(f'<tr style="background-color: {bg};">')
        html.append(f'<td style="padding: 8px; border: 1px solid #ddd; font-family: monospace; font-size: 0.85em;">{formatted["id"][:12]}...</td>')
        html.append(f'<td style="padding: 8px; border: 1px solid #ddd;">{formatted["type"]}</td>')
        html.append(f'<td style="padding: 8px; border: 1px solid #ddd; font-family: monospace; font-size: 0.85em;">{formatted["resourceId"][:20]}{"..." if len(formatted["resourceId"]) > 20 else ""}</td>')
        html.append(f'<td style="padding: 8px; border: 1px solid #ddd;">{formatted["userId"] or "-"}</td>')
        html.append(f'<td style="padding: 8px; border: 1px solid #ddd;">{formatted["groupId"] or "-"}</td>')
        html.append(f'<td style="padding: 8px; border: 1px solid #ddd;">{formatted["permissions"]}</td>')
        html.append('</tr>')
    
    html.append('</table>')
    return ''.join(html)


def on_query_resource_click(button):
    """Query authorizations by resource ID."""
    with output_resource:
        clear_output()
        
        resource_id = w_resource_id.value.strip()
        if not resource_id:
            print("❌ Please enter a resource ID")
            return
        
        print(f"🔍 Querying authorizations for resource: {resource_id}...")
        
        try:
            authorizations = Operaton.get(f"/authorization?resourceId={resource_id}")
            display(HTML(render_authorizations_table(authorizations)))
        except Exception as e:
            print(f"❌ Error: {e}")


def on_find_principals_click(button):
    """Find all principals (users/groups) with access to instances by business key."""
    with output_business_key:
        clear_output()
        
        business_key = w_business_key.value.strip()
        if not business_key:
            print("❌ Please enter a business key")
            return
        
        print(f"🔍 Finding principals for business key: {business_key}...")
        
        try:
            # Get process instances by business key
            instances = Operaton.get(f"/history/process-instance?processInstanceBusinessKey={business_key}")
            
            if not instances:
                print(f"⚠️ No process instances found with business key: {business_key}")
                return
            
            print(f"   Found {len(instances)} process instance(s)")
            
            # Collect all authorizations
            all_authorizations = []
            for instance in instances:
                auths = Operaton.get(f"/authorization?resourceId={instance['id']}")
                all_authorizations.extend(auths)
            
            # Extract unique principals
            users = set()
            groups = set()
            
            for auth in all_authorizations:
                if auth.get("userId"):
                    users.add(auth["userId"])
                if auth.get("groupId"):
                    groups.add(auth["groupId"])
            
            # Display results
            html = ['<h3>Principals with Access</h3>']
            
            if users:
                html.append('<h4>👤 Users:</h4><ul>')
                for user in sorted(users):
                    html.append(f'<li>{user}</li>')
                html.append('</ul>')
            else:
                html.append('<p><em>No user authorizations found</em></p>')
            
            if groups:
                html.append('<h4>👥 Groups:</h4><ul>')
                for group in sorted(groups):
                    html.append(f'<li>{group}</li>')
                html.append('</ul>')
            else:
                html.append('<p><em>No group authorizations found</em></p>')
            
            html.append(f'<p><strong>Total:</strong> {len(users)} user(s), {len(groups)} group(s) across {len(all_authorizations)} authorization(s)</p>')
            
            display(HTML(''.join(html)))
            
        except Exception as e:
            print(f"❌ Error: {e}")


def on_list_auths_click(button):
    """List all authorizations for instances by business key."""
    with output_business_key:
        clear_output()
        
        business_key = w_business_key.value.strip()
        if not business_key:
            print("❌ Please enter a business key")
            return
        
        print(f"🔍 Listing authorizations for business key: {business_key}...")
        
        try:
            # Get process instances by business key
            instances = Operaton.get(f"/history/process-instance?processInstanceBusinessKey={business_key}")
            
            if not instances:
                print(f"⚠️ No process instances found with business key: {business_key}")
                return
            
            print(f"   Found {len(instances)} process instance(s)")
            
            # Collect all authorizations
            all_authorizations = []
            for instance in instances:
                auths = Operaton.get(f"/authorization?resourceId={instance['id']}")
                all_authorizations.extend(auths)
            
            display(HTML(render_authorizations_table(all_authorizations)))
            
        except Exception as e:
            print(f"❌ Error: {e}")


def on_filter_auths_click(button):
    """Filter authorizations by user or group."""
    with output_filter:
        clear_output()
        
        business_key = w_filter_business_key.value.strip()
        user_id = w_filter_user.value.strip()
        group_id = w_filter_group.value.strip()
        
        if not business_key:
            print("❌ Please enter a business key")
            return
        
        if not user_id and not group_id:
            print("❌ Please enter at least a user ID or group ID to filter by")
            return
        
        print(f"🔍 Filtering authorizations...")
        
        try:
            # Get process instances by business key
            instances = Operaton.get(f"/history/process-instance?processInstanceBusinessKey={business_key}")
            
            if not instances:
                print(f"⚠️ No process instances found with business key: {business_key}")
                return
            
            # Collect filtered authorizations
            all_authorizations = []
            for instance in instances:
                if user_id:
                    auths = Operaton.get(f"/authorization?resourceId={instance['id']}&userIdIn={user_id}")
                    all_authorizations.extend(auths)
                if group_id:
                    auths = Operaton.get(f"/authorization?resourceId={instance['id']}&groupIdIn={group_id}")
                    all_authorizations.extend(auths)
            
            # Remove duplicates (by ID)
            seen_ids = set()
            unique_auths = []
            for auth in all_authorizations:
                if auth['id'] not in seen_ids:
                    seen_ids.add(auth['id'])
                    unique_auths.append(auth)
            
            filter_desc = []
            if user_id:
                filter_desc.append(f"user={user_id}")
            if group_id:
                filter_desc.append(f"group={group_id}")
            
            display(HTML(render_authorizations_table(unique_auths, f"Filtered Authorizations ({', '.join(filter_desc)})")))
            
        except Exception as e:
            print(f"❌ Error: {e}")


def on_preview_delete_click(button):
    """Preview authorizations to be deleted."""
    with output_delete:
        clear_output()
        
        business_key = w_delete_business_key.value.strip()
        user_id = w_delete_user.value.strip()
        group_id = w_delete_group.value.strip()
        
        if not business_key:
            print("❌ Please enter a business key")
            btn_delete_auths.disabled = True
            return
        
        if not user_id and not group_id:
            print("❌ Please enter at least a user ID or group ID to filter deletions")
            btn_delete_auths.disabled = True
            return
        
        print(f"🔍 Finding authorizations to delete...")
        
        try:
            # Get process instances by business key
            instances = Operaton.get(f"/history/process-instance?processInstanceBusinessKey={business_key}")
            
            if not instances:
                print(f"⚠️ No process instances found with business key: {business_key}")
                btn_delete_auths.disabled = True
                return
            
            # Collect filtered authorizations
            all_authorizations = []
            for instance in instances:
                if user_id:
                    auths = Operaton.get(f"/authorization?resourceId={instance['id']}&userIdIn={user_id}")
                    all_authorizations.extend(auths)
                if group_id:
                    auths = Operaton.get(f"/authorization?resourceId={instance['id']}&groupIdIn={group_id}")
                    all_authorizations.extend(auths)
            
            # Remove duplicates
            seen_ids = set()
            unique_auths = []
            for auth in all_authorizations:
                if auth['id'] not in seen_ids:
                    seen_ids.add(auth['id'])
                    unique_auths.append(auth)
            
            delete_state['authorizations_to_delete'] = unique_auths
            
            if not unique_auths:
                print("✅ No authorizations found matching the criteria")
                btn_delete_auths.disabled = True
                return
            
            display(HTML(render_authorizations_table(unique_auths, "⚠️ Authorizations to be DELETED")))
            print(f"\n⚠️ {len(unique_auths)} authorization(s) will be deleted!")
            print("   Check the confirmation box and click 'Delete Authorizations' to proceed.")
            
            btn_delete_auths.disabled = False
            
        except Exception as e:
            print(f"❌ Error: {e}")
            btn_delete_auths.disabled = True


def on_delete_auths_click(button):
    """Delete the previewed authorizations."""
    with output_delete:
        if not w_confirm_delete.value:
            print("❌ Please check the confirmation box before deleting")
            return
        
        authorizations = delete_state.get('authorizations_to_delete', [])
        
        if not authorizations:
            print("❌ No authorizations to delete. Please preview first.")
            return
        
        print(f"\n🗑️ Deleting {len(authorizations)} authorization(s)...")
        
        success_count = 0
        error_count = 0
        
        for auth in authorizations:
            try:
                Operaton.delete(f"/authorization/{auth['id']}")
                print(f"   ✅ Deleted: {auth['id'][:12]}...")
                success_count += 1
            except Exception as e:
                print(f"   ❌ Failed to delete {auth['id'][:12]}...: {e}")
                error_count += 1
        
        print(f"\n📊 Summary: {success_count} deleted, {error_count} failed")
        
        # Reset state
        delete_state['authorizations_to_delete'] = []
        w_confirm_delete.value = False
        btn_delete_auths.disabled = True


# Attach handlers
btn_query_resource.on_click(on_query_resource_click)
btn_find_principals.on_click(on_find_principals_click)
btn_list_auths.on_click(on_list_auths_click)
btn_filter_auths.on_click(on_filter_auths_click)
btn_preview_delete.on_click(on_preview_delete_click)
btn_delete_auths.on_click(on_delete_auths_click)

print("✅ Event handlers attached")

In [ ]:
# === DISPLAY UI ===

# Tab 1: Query by Resource ID
tab_resource = widgets.VBox([
    widgets.HTML('<h3>🔍 Query by Resource ID</h3>'),
    widgets.HTML('<p>Find all authorizations for a specific resource (e.g., process instance ID, task ID).</p>'),
    w_resource_id,
    btn_query_resource,
    output_resource
])

# Tab 2: Query by Business Key
tab_business_key = widgets.VBox([
    widgets.HTML('<h3>🏷️ Query by Business Key</h3>'),
    widgets.HTML('<p>Find authorizations for all process instances with a specific business key.</p>'),
    w_business_key,
    widgets.HBox([btn_find_principals, btn_list_auths]),
    output_business_key
])

# Tab 3: Filter by User/Group
tab_filter = widgets.VBox([
    widgets.HTML('<h3>🎯 Filter by User/Group</h3>'),
    widgets.HTML('<p>Find authorizations for specific users or groups on process instances.</p>'),
    w_filter_business_key,
    w_filter_user,
    w_filter_group,
    btn_filter_auths,
    output_filter
])

# Tab 4: Delete Authorizations
tab_delete = widgets.VBox([
    widgets.HTML('<h3>🗑️ Delete Authorizations</h3>'),
    widgets.HTML('<p><strong>⚠️ Warning:</strong> This will permanently delete authorizations. Use with caution!</p>'),
    w_delete_business_key,
    w_delete_user,
    w_delete_group,
    widgets.HBox([btn_preview_delete, btn_delete_auths]),
    w_confirm_delete,
    output_delete
])

# Create tabs
tabs = widgets.Tab([tab_resource, tab_business_key, tab_filter, tab_delete])
tabs.set_title(0, '🔍 By Resource')
tabs.set_title(1, '🏷️ By Business Key')
tabs.set_title(2, '🎯 Filter')
tabs.set_title(3, '🗑️ Delete')

# Main UI
display(widgets.VBox([
    widgets.HTML('<h1>🔐 Authorization Audit Tool</h1>'),
    widgets.HTML('<p>Query, audit, and manage Operaton/Camunda authorizations.</p>'),
    widgets.HTML('<hr>'),
    tabs
]))